In [1]:
# Required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from statsmodels.stats import diagnostic
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
data_folder = './MovieSummaries/MovieSummaries/'
#paths to files
plot_summaries_path = data_folder + 'plot_summaries.txt'
movie_metadata_path = data_folder + 'movie.metadata.tsv'
character_metadata_path = data_folder + 'character.metadata.tsv'

# load the data
# 1. Plot summaries data
plot_summaries_df = pd.read_csv(plot_summaries_path, delimiter='\t', names=['wikipedia_movie_id', 'plot_summary'], 
                                 encoding='utf-8')

# 2. Movie metadata
movie_metadata_df = pd.read_csv(movie_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 
                                                                            'movie_name', 'release_date', 'box_office_revenue',
                                                                            'runtime', 'languages', 'countries', 'genres'], 
                                 encoding='utf-8')

# 3. Character metadata
character_metadata_df = pd.read_csv(character_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 
                                                                                    'actor_dob', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 
                                                                                    'actor_age_at_release', 'freebase_character_actor_map_id', 'freebase_character_id', 
                                                                                    'freebase_actor_id'], 
                                     encoding='utf-8')


In [ ]:
print("Size of plot_summaries_df :", plot_summaries_df.shape)
print("Size of movie_metadata_df :", movie_metadata_df.shape)
print("Size of character_metadata_df :", character_metadata_df.shape)


In [ ]:
movie_metadata_df.head()

In [87]:
unique_ethnicities = character_metadata_df['actor_ethnicity'].dropna().unique()


In [88]:
import csv

ethnicities_csv_path = 'ethnicities_with_skin_color.csv'

with open(ethnicities_csv_path, mode='r') as file:
    reader = csv.DictReader(file)
    ethnicity_skin_color_map = {row['Ethnicity']: row['Skin_Color'] for row in reader}

skin_color_dict = dict(zip(unique_ethnicities, list(ethnicity_skin_color_map.values())))

character_metadata_df['skin_color'] = character_metadata_df['actor_ethnicity'].map(skin_color_dict)
character_metadata_df['skin_color'] = character_metadata_df['skin_color'].apply(lambda s: s if type(s) != str or ('/' not in s or s.startswith('White/Middle Eastern')) else 'Mixed')

In [ ]:
# Convert release_date to datetime
movie_metadata_df['release_date'] = pd.to_datetime(movie_metadata_df['release_date'], errors='coerce')

# Extract year from release_date
movie_metadata_df['release_year'] = movie_metadata_df['release_date'].dt.year

movie_metadata_df['box_office_revenue'] = movie_metadata_df['box_office_revenue'].fillna(0)

# Group by release_year and get the movie with the highest box_office_revenue for each year
highest_box_office_per_year = movie_metadata_df.loc[movie_metadata_df.groupby('release_year')['box_office_revenue'].idxmax()]

# Drop rows with NaN values in box_office_revenue
highest_box_office_per_year = highest_box_office_per_year.dropna(subset=['box_office_revenue'])

# Display the result
# print(highest_box_office_per_year[['release_year', 'movie_name', 'box_office_revenue']])
# Merge character metadata with movie metadata
merged_df = pd.merge(character_metadata_df, highest_box_office_per_year[['wikipedia_movie_id', 'release_date']], on='wikipedia_movie_id', how='left')

print(highest_box_office_per_year.release_year)

# Map ethnicities to skin colors
#merged_df['skin_color'] = merged_df['actor_ethnicity'].map(skin_color_dict)

# Extract year from release_date
merged_df['release_year'] = pd.to_datetime(merged_df['release_date_x'], errors='coerce').dt.year
#merged_df['skin_color'].fillna('Unknown', inplace=True)

# Group by year and skin color, then calculate the ratio of each ethnicity per film
ethnicity_per_year = merged_df.groupby(['release_year', 'skin_color']).size().unstack(fill_value=0)

# Calculate the ratio of each ethnicity
ethnicity_per_year = ethnicity_per_year.div(ethnicity_per_year.sum(axis=1), axis=0)

# Calculate the total number of films per year
#total_films_per_year = merged_df.groupby('release_year')['wikipedia_movie_id'].nunique()

# Calculate the ratio of each ethnicity per film per year
ethnicity_ratio_per_year = ethnicity_per_year#.div(total_films_per_year, axis=0)

# Display the result
# Plot the ratio of each ethnicity per film per year
plt.figure(figsize=(14, 8))
ethnicity_ratio_per_year.plot(kind='area', stacked=True, colormap='tab20', alpha=0.8)
plt.title('Ratio of Each Ethnicity per Film per Year')
plt.xlabel('Year')
plt.ylabel('Ratio of Ethnicity per Film')
plt.legend(title='Ethnicity', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Display the result
# Plot the ratio of each ethnicity per film per year
# Plot the ratio of each ethnicity per film per year in subplots
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20), sharex=True, sharey=True)
axes = axes.flatten()

for i, ethnicity in enumerate(ethnicity_ratio_per_year.columns):
    sns.lineplot(x=ethnicity_ratio_per_year.index, y=ethnicity_ratio_per_year[ethnicity], ax=axes[i])
    axes[i].set_title(f'Ethnicity: {ethnicity}')
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('Ratio')

plt.tight_layout()
plt.show()

In [ ]:

# first few rows of each dataframe to understand
print("Plot Summaries DataFrame:")
print(plot_summaries_df.head(), "\n")

print("Movie Metadata DataFrame:")
print(movie_metadata_df.head(), "\n")

print("Character Metadata DataFrame:")
print(character_metadata_df.head(), "\n")



In [ ]:
headers = movie_metadata_df.columns.tolist()

print("Headers of the movie_metadata_df:")
print(headers)
headers = plot_summaries_df.columns.tolist()

print("Headers of the plot_summaries_df:")
print(headers)
headers = character_metadata_df.columns.tolist()

print("Headers of the character_metadata_df:")
print(headers)

In [ ]:

# missing values
print("Missing values in Plot Summaries DataFrame:")
print(plot_summaries_df.isnull().sum(), "\n")

print("Missing values in Movie Metadata DataFrame:")
print(movie_metadata_df.isnull().sum(), "\n")

print("Missing values in Character Metadata DataFrame:")
print(character_metadata_df.isnull().sum(), "\n")
#

In [ ]:
for df in [character_metadata_df, movie_metadata_df, plot_summaries_df]:
    df['wikipedia_movie_id'] = pd.to_numeric(df['wikipedia_movie_id'], errors='coerce').fillna(-1).astype(int)

# Change data types
plot_summaries_df['plot_summary'] = plot_summaries_df['plot_summary'].astype('string')  
movie_metadata_df['movie_name'] = movie_metadata_df['movie_name'].astype('string')  
character_metadata_df['actor_name'] = character_metadata_df['actor_name'].astype('string')  
character_metadata_df['character_name'] = character_metadata_df['character_name'].astype('string')  
character_metadata_df['actor_gender'] = character_metadata_df['actor_gender'].astype('string')  
movie_metadata_df['freebase_movie_id'] = movie_metadata_df['freebase_movie_id'].astype('string')  
character_metadata_df['freebase_character_actor_map_id'] = character_metadata_df['freebase_character_actor_map_id'].astype('string')  
character_metadata_df['freebase_character_id'] = character_metadata_df['freebase_character_id'].astype('string')  
character_metadata_df['freebase_actor_id'] = character_metadata_df['freebase_actor_id'].astype('string')  

# Convert to float
character_metadata_df['actor_age_at_release'] = pd.to_numeric(character_metadata_df['actor_age_at_release'], errors='coerce')




print("Data types in character_metadata_df:")
print(character_metadata_df.dtypes)
print("\n Data types in movie_metadata_df:")
print(movie_metadata_df.dtypes)
print("\n Data types in plot_summaries_df:")
print(plot_summaries_df.dtypes)


In [ ]:
#1st way of extracting
import re
def extract_dictionnaries_from_string(country_string):
    if isinstance(country_string, str) and country_string != '{}':
        # Use regex to find all country names (text inside double quotes after the colon)
        return ', '.join(re.findall(r'": "([^"]+)"', country_string))  # Extract country names after ": "
    return np.nan  # Return NaN for empty strings or {}

# Apply the function to extract country names
movie_metadata_df['Cleaned_countries'] = movie_metadata_df['countries'].apply(extract_dictionnaries_from_string)

# Find unique countries
# Flatten the list of country names and extract unique values
unique_countries = pd.Series([country for countries in movie_metadata_df['Cleaned_countries'].dropna() 
                              for country in countries.split(', ')]).unique()

# Show results
print("Cleaned DataFrame:")
print(movie_metadata_df[['countries', 'Cleaned_countries']])
print("\nUnique countries:")
print(unique_countries)

In [ ]:
# Apply the function to extract genres names
movie_metadata_df['Cleaned_genres'] = movie_metadata_df['genres'].apply(extract_dictionnaries_from_string)

# Find unique countries
# Flatten the list of country names and extract unique values
unique_genres = pd.Series([genre for genres in movie_metadata_df['Cleaned_genres'].dropna() 
                              for genre in genres.split(', ')]).unique()

# Show results
print("Cleaned DataFrame:")
print(movie_metadata_df[['genres', 'Cleaned_genres']])
print("\nUnique genres:")
print(unique_genres)

In [ ]:
# Apply the function to extract languages names
movie_metadata_df['Cleaned_languages'] = movie_metadata_df['languages'].apply(extract_dictionnaries_from_string)

# Find unique countries
# Flatten the list of country names and extract unique values
unique_languages = pd.Series([language for languages in movie_metadata_df['Cleaned_languages'].dropna() 
                              for language in languages.split(', ')]).unique()

# Show results
print("Cleaned DataFrame:")
print(movie_metadata_df[['languages', 'Cleaned_languages']])
print("\nUnique languages:")
print(unique_languages)

In [ ]:
import json
# Convert to json
movie_metadata_df['countries'] = movie_metadata_df['countries'].apply(json.loads)
movie_metadata_df['languages'] = movie_metadata_df['languages'].apply(json.loads)  
movie_metadata_df['genres'] = movie_metadata_df['genres'].apply(json.loads) 

#2nd way of extracting
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')


def extract_values(value):
    if isinstance(value, dict):
        return list(value.values())
    elif isinstance(value, str):
        return [value]
    return []
# extraction of values for genres
movie_metadata_df['genres'] = movie_metadata_df['genres'].apply(extract_values)

unique_genres = set(genre for genres in movie_metadata_df['genres'] for genre in genres)

for genre in unique_genres:
    movie_metadata_df[genre] = movie_metadata_df['genres'].apply(lambda x: 1 if genre in x else 0)

print(unique_genres)

In [ ]:

# extraction of values for language
movie_metadata_df['languages'] = movie_metadata_df['languages'].apply(extract_values)

unique_languages = set(language for languages in movie_metadata_df['languages'] for language in languages)

for language in unique_languages:
    movie_metadata_df[language] = movie_metadata_df['languages'].apply(lambda x: 1 if language in x else 0)

print(unique_languages)

In [ ]:
# extraction of values for countries
movie_metadata_df['countries'] = movie_metadata_df['countries'].apply(extract_values)

unique_countries = set(country for countries in movie_metadata_df['countries'] for country in countries)

for country in unique_countries:
    movie_metadata_df[country] = movie_metadata_df['countries'].apply(lambda x: 1 if country in x else 0)

print(unique_countries)

In [14]:
# Remove duplicates
plot_summaries_df.drop_duplicates(subset='wikipedia_movie_id', inplace=True)
movie_metadata_df.drop_duplicates(subset='wikipedia_movie_id', inplace=True)

# Convert numerical columns to numeric types (coerce errors to handle non-numeric entries)
movie_metadata_df['box_office_revenue'] = pd.to_numeric(movie_metadata_df['box_office_revenue'], errors='coerce')
movie_metadata_df['runtime'] = pd.to_numeric(movie_metadata_df['runtime'], errors='coerce')
character_metadata_df['actor_height'] = pd.to_numeric(character_metadata_df['actor_height'], errors='coerce')
character_metadata_df['actor_age_at_release'] = pd.to_numeric(character_metadata_df['actor_age_at_release'], errors='coerce')


In [ ]:
# to understand the distributions
print("Summary statistics for Movie Metadata:")
print(movie_metadata_df[['box_office_revenue', 'runtime']].describe(), "\n")

print("Summary statistics for Character Metadata:")
print(character_metadata_df[['actor_height', 'actor_age_at_release']].describe(), "\n")


In [ ]:
over_possible_duration = (movie_metadata_df['runtime']> 420).sum()
print( over_possible_duration, "movies are over 7 hours runtime ")

movie_metadata_df['runtime'] = movie_metadata_df['runtime'].where(movie_metadata_df['runtime']<420, np.nan)
character_metadata_df['actor_age_at_release'] = character_metadata_df['actor_age_at_release'].where(character_metadata_df['actor_age_at_release']>0, np.nan)

big_revenue= (movie_metadata_df['box_office_revenue']>1e9).sum()
print( big_revenue , "movies made more than a billion revenue")

In [ ]:

#histograms 
plt.figure(figsize=(10, 5))
sns.histplot(movie_metadata_df['box_office_revenue'].dropna(), bins=50, kde=True)
plt.title('Box Office Revenue Distribution')
plt.xlabel('Box Office Revenue')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(10, 5))
sns.histplot(movie_metadata_df['runtime'].dropna(), bins=30, kde=True)
plt.title('Runtime Distribution')
plt.xlabel('Runtime (minutes)')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(10, 5))
sns.histplot(character_metadata_df['actor_age_at_release'].dropna(), bins=30, kde=True)
plt.title('Actor Age at Release Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# 1. Number of films per language
language_counts = movie_metadata_df[list(unique_languages)].sum()

sorted_language_counts  = language_counts.sort_values(ascending=False)
top_languages = sorted_language_counts.head(10)


# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x=top_languages.index, y=top_languages.values)
plt.title('Number of Films per Language')
plt.xticks(rotation=90)
plt.xlabel('Language')
plt.ylabel('Number of Films')
plt.show()



In [ ]:

# 2. Number of films per country
countries_counts = movie_metadata_df[list(unique_countries)].sum()

sorted_countries_counts  = countries_counts.sort_values(ascending=False)
top_countries = sorted_countries_counts.head(5)


# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x=top_countries.index, y=top_countries.values)
plt.title('Number of Films per Country')
plt.xticks(rotation=90)
plt.xlabel('Country')
plt.ylabel('Number of Films')
plt.show()


In [ ]:
# 3. Number of films per genre
genres_counts = movie_metadata_df[list(unique_genres)].sum()

sorted_genres_counts  = genres_counts.sort_values(ascending=False)
top_genres = sorted_genres_counts.head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title('Number of Films per Genre')
plt.xticks(rotation=90)
plt.xlabel('Genre')
plt.ylabel('Number of Films')
plt.show()


In [ ]:
#Getting years and extracting datas : 1st way

def process_release_date(df, column_to_load, new_column_name):
    def extract_year(value):
        if pd.isna(value):
            return np.nan
        try:
            # Try to convert to datetime first
            return pd.to_datetime(value).year
        except:
            # If it fails, check if it's a year
            if isinstance(value, (int, float)) or (isinstance(value, str) and value.isdigit()):
                return int(float(value))
            return np.nan

    # Apply the function to the 'Release_date' column
    df[new_column_name] = df[column_to_load].apply(extract_year)
    
    # Ensure the column is of type Int64 to handle NaN values
    df[new_column_name] = df[new_column_name].astype('Int64')
    




process_release_date(movie_metadata_df, 'release_date' ,'release_year')


# Display the first few rows to verify the changes
print(movie_metadata_df['release_year'].head())

In [ ]:
#takes too much time

# process_release_date(character_metadata_df, 'actor_dob' ,'actor_year_ob')
# print(character_metadata_df['actor_year_ob'].head())

In [ ]:
#takes too much time

# process_release_date(character_metadata_df, 'release_date' ,'release_year')
# print(character_metadata_df['release_year'].head())

In [ ]:
#Getting years and extracting datas : 2nd way


# Convert date columns to datetime format for consistency
movie_metadata_df['release_date'] = pd.to_datetime(movie_metadata_df['release_date'], errors='coerce')
character_metadata_df['release_date'] = pd.to_datetime(character_metadata_df['release_date'], errors='coerce')
character_metadata_df['actor_dob'] = pd.to_datetime(character_metadata_df['actor_dob'], errors='coerce')

# 4. Distribution of actor's date of birth (dob)
plt.figure(figsize=(12, 6))
sns.histplot(character_metadata_df['actor_dob'], bins=30, kde=True)
plt.title('Distribution of Actor Date of Birth')
plt.xlabel('Actor Date of Birth')
plt.ylabel('Frequency')
plt.show()


In [ ]:

# 5. Distribution of actor gender
actor_gender_counts = character_metadata_df['actor_gender'].value_counts()
plt.figure(figsize=(6, 6))
sns.barplot(x=actor_gender_counts.index, y=actor_gender_counts.values)
plt.title('Distribution of Actor Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Actors')
plt.show()


In [ ]:


# 6. Distribution of actor age at release
character_metadata_df['actor_age_at_release'] = (character_metadata_df['release_date'] - character_metadata_df['actor_dob']).dt.days / 365.25
plt.figure(figsize=(12, 6))
sns.histplot(character_metadata_df['actor_age_at_release'], bins=50, kde=True)
plt.title('Distribution of Actor Age at Release')
plt.xlabel('Actor Age at Release')
plt.ylabel('Frequency')
plt.show()


In [ ]:


# 7. Time series analysis


# 7.1 Number of women in movies over time
women_counts_over_time = character_metadata_df[character_metadata_df['actor_gender'] == 'F'].groupby(character_metadata_df['release_date'].dt.year).size()
plt.figure(figsize=(12, 6))
sns.lineplot(x=women_counts_over_time.index, y=women_counts_over_time.values)
plt.title('Number of Women in Movies Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Women')
plt.show()

# 7.2 Runtime of movies over time
runtime_over_time = movie_metadata_df.groupby(movie_metadata_df['release_date'].dt.year)['runtime'].mean()
plt.figure(figsize=(12, 6))
sns.lineplot(x=runtime_over_time.index, y=runtime_over_time.values)
plt.title('Average Runtime of Movies Over Time')
plt.xlabel('Year')
plt.ylabel('Average Runtime (minutes)')
plt.show()

# 7.3 Number of languages per film over time
# Create a new column for the number of languages

movie_metadata_df['num_languages'] = movie_metadata_df['languages'].str.len()
languages_per_film_over_time = movie_metadata_df.groupby(movie_metadata_df['release_date'].dt.year)['num_languages'].mean()
plt.figure(figsize=(12, 6))
sns.lineplot(x=languages_per_film_over_time.index, y=languages_per_film_over_time.values)
plt.title('Average Number of Languages per Film Over Time')
plt.xlabel('Year')
plt.ylabel('Average Number of Languages')
plt.show()



In [30]:
top_countries_index = top_countries.index


# 4. Tracer le nombre de films par pays au fil du temps
filtered_df = movie_metadata_df[movie_metadata_df['countries'].apply(lambda x: any(country in x for country in top_countries_index))]

# Convertir les listes de pays en lignes distinctes pour chaque pays dans chaque film
exploded_df = filtered_df.explode('countries')

# Filtrer pour conserver uniquement les lignes avec les pays du top
exploded_df = exploded_df[exploded_df['countries'].isin(top_countries_index)]

# Calculer le nombre de films par pays au fil du temps
films_per_country_over_time = (exploded_df.groupby([exploded_df['release_date'].dt.year, 'countries'])
                               .size()
                               .unstack(fill_value=0))



In [ ]:
# Tracer le nombre de films par pays au fil du temps
plt.figure(figsize=(16, 10))
films_per_country_over_time.plot(kind='line', marker='.', linewidth=1)
plt.title('Number of Films per Country Over Time (Top 5 Countries)')
plt.xlabel('Year')
plt.ylabel('Number of Films')
# plt.yscale('log')
plt.legend(title='Countries', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
year_occurrences = movie_metadata_df['release_date'].dt.year.value_counts().sort_index()

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.bar(year_occurrences.index, year_occurrences.values)
plt.xlabel('Year')
plt.ylabel('Number of movies in total')
plt.title('Number of movies by Year')
plt.xticks(rotation=0)  # Rotate x-axis labels if needed
plt.show()